In [1]:
import pandas as pd

# 讀取 CSV 文件
csv_path = "C:/PYTHON/國科會/csv_json/刑法.csv"
df = pd.read_csv(csv_path, encoding='utf-8-sig')

# 檢查每列的 NaN 值數量
nan_count = df.isna().sum()

# 顯示有 NaN 值的列及其數量
print("每列的 NaN 值數量:")
print(nan_count)

# 顯示有 NaN 值的行
nan_rows = df[df.isna().any(axis=1)]
print("\n含有 NaN 值的行:")
print(nan_rows)


每列的 NaN 值數量:
LawLevel           0
ArticleNo          0
ArticleContent    10
dtype: int64

含有 NaN 值的行:
    LawLevel ArticleNo ArticleContent
91        刑法      第92條            NaN
118       刑法     第119條            NaN
235       刑法     第236條            NaN
244       刑法     第245條            NaN
286       刑法     第287條            NaN
307       刑法     第308條            NaN
318       刑法     第319條            NaN
337       刑法     第338條            NaN
342       刑法     第343條            NaN
356       刑法     第357條            NaN


In [2]:
from neo4j import GraphDatabase
import pandas as pd
import openai
import os
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
# 設定 Neo4j 連線
uri = "bolt://localhost:7687"
username = ""
password = ""
driver = GraphDatabase.driver(uri, auth=(username, password))

# 使用 OpenAI 生成文本向量的函數
os.environ['AzureOpenAIEmbeddings_KEY'] = ""
os.environ["AzureOpenAIEmbeddings_ENDPOINT"] = ""
os.environ['AzureChatOpenAI_KEY'] = ""
os.environ["AzureChatOpenAI_ENDPOINT"] = ""
# 初始化 Azure OpenAI Embedding 和 Chat 服務
embedding_model = AzureOpenAIEmbeddings(
    api_key=os.environ['AzureOpenAIEmbeddings_KEY'],  # 設定 Azure API 金鑰
    api_version="2023-05-15",  # 設定 Azure API 版本
    model="text-embedding-ada-002",  # 設定模型名稱
    azure_endpoint=os.environ['AzureOpenAIEmbeddings_ENDPOINT']  # 設定 Azure 端點
)

# 使用 Azure OpenAI 生成文本向量
def generate_vector(text: str):
    """使用 Azure OpenAI 生成文本的向量"""
    # 生成文本的向量
    vector = embedding_model.embed_query(text)  # 使用 embed_query 生成向量
    return vector

# 定義數據插入函數
def import_data(tx, row):
    # 生成 Article 節點的向量
    article_vector = generate_vector(row['ArticleContent'])
    
    # 分割 CKIP Text 並對每個詞生成向量
    words = row['CKIP Text'].split("||")
    word_vectors = {word: generate_vector(word) for word in words}

    # 插入數據到 Neo4j
    query = """
    MERGE (law:Law {name: "刑法"})
    MERGE (article:Article {number: $ArticleNo, content: $ArticleContent, vector: $article_vector})
    MERGE (law)-[:Include]->(article)
    WITH article, $word_data AS word_data
    UNWIND word_data AS word_map
    MERGE (ckip:CKIP {text: word_map.word})
    SET ckip.vector = word_map.vector
    MERGE (ckip)-[:belong]->(article)
    """
    
    # 將詞彙及其向量作為參數傳遞給查詢
    tx.run(query, 
           ArticleNo=row['ArticleNo'], 
           ArticleContent=row['ArticleContent'], 
           article_vector=article_vector,
           word_data=[{'word': word, 'vector': vector} for word, vector in word_vectors.items()])

# 讀取本地 CSV 文件
csv_path = r"C:\PYTHON\自主學習\csv_json\刑法_去除停用詞.csv"
df = pd.read_csv(csv_path, encoding='utf-8-sig')

# 將數據逐行寫入 Neo4j
with driver.session() as session:
    for index, row in df.iterrows():
        session.write_transaction(import_data, row)

print("數據插入完成！")

C:\Users\sheng\AppData\Local\Temp\ipykernel_16940\3256547511.py:67: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(import_data, row)


數據插入完成！
